### Preparing and train yolo end to end
#### Train with even-smaller-yolotinyv3, so call "supertiny"

In [ ]:
import os,cv2,matplotlib.pyplot as plt

### Prepare data, create symlink to the original image & label

In [5]:
root_dir = '/mnt/disk2/datasets/Wider-mask-detection/'

In [16]:
img_paths = []
lbl_paths = []
for root,_,files in os.walk(root_dir):
    for f in files:
        if '.jpg' in f:
            lbl_path= root.replace('/images/','/labels/')+'/'+f.replace('.jpg','.txt')
            if os.path.exists(lbl_path):
                lbl_paths.append (lbl_path)
                img_paths.append (root+'/'+f)
            

In [17]:
len (lbl_paths)

16098

In [18]:
cls = {}
for path in lbl_paths:
    lines = open(path,'rt').read().strip().split('\n')
    for l in lines:
        p = l.split()
        cls[p[0]]= 0

In [20]:
cls

{'0': 0, '1': 0}

### Create folder for storing symlinks to images & labels

In [21]:
!mkdir /home/namhv/source_code/darknet3/data/mask_kaggle/

In [25]:
for img_path,lbl_path in zip(img_paths,lbl_paths):
    try:
        os.symlink(img_path,'/home/namhv/source_code/darknet3/data/mask_kaggle/'+os.path.basename(img_path))
        os.symlink(lbl_path,'/home/namhv/source_code/darknet3/data/mask_kaggle/'+os.path.basename(lbl_path))
    except:
        pass
#     break

### Create train/validate files

In [45]:
writer = open('/home/namhv/source_code/darknet3/data/kaggle_face_train.txt','wt')
for img_path,lbl_path in zip(img_paths[:13000],lbl_paths[:13000]):
    writer.write('data/mask_kaggle/'+os.path.basename(img_path)+'\n')
writer.close()

In [32]:
writer = open('/home/namhv/source_code/darknet3/data/kaggle_face_val.txt','wt')
for img_path,lbl_path in zip(img_paths[13000:],lbl_paths[13000:]):
    writer.write('data/mask_kaggle/'+os.path.basename(img_path)+'\n')
writer.close()

### Create cfg supertiny v3
set the classes=2 value in the line 124 and  filters=(classes + 5)x3  in line 172.

In [39]:
!rm /home/namhv/source_code/darknet3/cfg/kaggle_face_supertiny3.cfg

In [44]:
%%writefile /home/namhv/source_code/darknet3/cfg/kaggle_face_supertiny3.cfg
[net]
# Testing
batch=24
subdivisions=1
# Training
batch=2048
subdivisions=1
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=16
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=16
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=1

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

###########

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=27
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=4
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 8

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
# change this value: filters = (classes+5)*3
filters=21
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
# change this value
classes=2
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1


Overwriting /home/namhv/source_code/darknet3/cfg/kaggle_face_supertiny3.cfg


In [28]:
%%writefile /home/namhv/source_code/darknet3/cfg/kaggle_face_supertiny3.names
normal
masked

Writing /home/namhv/source_code/darknet3/cfg/kaggle_face_supertiny3.names


In [33]:
!mkdir /home/namhv/source_code/darknet3/backup/kaggle_face/

In [42]:
!rm /home/namhv/source_code/darknet3/data/kaggle_face_train.txt

In [43]:
%%writefile /home/namhv/source_code/darknet3/cfg/kaggle_face_supertiny3.data
classes= 2
train  = data/kaggle_face_train.txt
# valid  = data/head/test.txt
names = cfg/kaggle_face_supertiny3.names
backup = backup/kaggle_face/

Writing /home/namhv/source_code/darknet3/cfg/kaggle_face_supertiny3.data


### Start training

In [ ]:
!./darknet detector train cfg/kaggle_face_supertiny3.data cfg/kaggle_face_supertiny3.cfg

### Test model

In [ ]:
!./darknet -nogpu detector test \
cfg/kaggle_face_supertiny3.data \
cfg/kaggle_face_supertiny3.cfg \
backup/kaggle_face/kaggle_face_supertiny3_1000.weights \
/data/dtcb_data/img_1036.jpg